In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import requirements

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 54.3 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 895 kB 42.9 MB/s 
     |████████████████████████████████| 6.5 MB 51.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import os
import pdb
import argparse
from dataclasses import dataclass, field
from typing import Optional
from collections import defaultdict

import torch
from torch.nn.utils.rnn import pad_sequence

import numpy as np
from tqdm import tqdm, trange

from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    AutoConfig,
    AdamW
)

# 1. Preprocess

In [ ]:
def make_id_file(task, tokenizer):
    def make_data_strings(file_name):
        data_strings = []
        with open(os.path.join(file_name), 'r', encoding='utf-8') as f:
            id_file_data = [tokenizer.encode(line.lower()) for line in f.readlines()]
        for item in id_file_data:
            data_strings.append(' '.join([str(k) for k in item]))
        return data_strings
    
    print('it will take some times...')
    train_pos = make_data_strings('sentiment.train.1')
    train_neg = make_data_strings('sentiment.train.0')
    dev_pos = make_data_strings('sentiment.dev.1')
    dev_neg = make_data_strings('sentiment.dev.0')

    print('make id file finished!')
    return train_pos, train_neg, dev_pos, dev_neg

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sentiment.dev.0 to sentiment.dev.0
Saving sentiment.dev.1 to sentiment.dev.1
Saving sentiment.train.0 to sentiment.train.0
Saving sentiment.train.1 to sentiment.train.1
Saving test_no_label.csv to test_no_label.csv


In [ ]:
!ls

drive	     sentiment.dev.0  sentiment.train.0  test_no_label.csv
sample_data  sentiment.dev.1  sentiment.train.1


In [ ]:
train_pos, train_neg, dev_pos, dev_neg = make_id_file('yelp', tokenizer)

it will take some times...
make id file finished!


In [ ]:
train_pos[:10]

['101 6581 2833 1012 102',
 '101 21688 8013 2326 1012 102',
 '101 2027 2036 2031 3679 19247 1998 3256 6949 2029 2003 2428 2204 1012 102',
 '101 2009 1005 1055 1037 2204 15174 2098 7570 22974 2063 1012 102',
 '101 1996 3095 2003 5379 1012 102',
 '101 2204 3347 2833 1012 102',
 '101 2204 2326 1012 102',
 '101 11350 1997 2154 2003 25628 1998 7167 1997 19247 1012 102',
 '101 2307 2173 2005 6265 2030 3347 27962 1998 5404 1012 102',
 '101 1996 2047 2846 3504 6429 1012 102']

In [ ]:
class SentimentDataset(object):
    def __init__(self, tokenizer, pos, neg):
        self.tokenizer = tokenizer
        self.data = []
        self.label = []

        for pos_sent in pos:
            self.data += [self._cast_to_int(pos_sent.strip().split())]
            self.label += [[1]]
        for neg_sent in neg:
            self.data += [self._cast_to_int(neg_sent.strip().split())]
            self.label += [[0]]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample), np.array(self.label[index])

In [ ]:
train_dataset = SentimentDataset(tokenizer, train_pos, train_neg)
dev_dataset = SentimentDataset(tokenizer, dev_pos, dev_neg)

In [ ]:
for i, item in enumerate(train_dataset):
    print(item)
    if i == 10:
        break

(array([ 101, 6581, 2833, 1012,  102]), array([1]))
(array([  101, 21688,  8013,  2326,  1012,   102]), array([1]))
(array([  101,  2027,  2036,  2031,  3679, 19247,  1998,  3256,  6949,
        2029,  2003,  2428,  2204,  1012,   102]), array([1]))
(array([  101,  2009,  1005,  1055,  1037,  2204, 15174,  2098,  7570,
       22974,  2063,  1012,   102]), array([1]))
(array([ 101, 1996, 3095, 2003, 5379, 1012,  102]), array([1]))
(array([ 101, 2204, 3347, 2833, 1012,  102]), array([1]))
(array([ 101, 2204, 2326, 1012,  102]), array([1]))
(array([  101, 11350,  1997,  2154,  2003, 25628,  1998,  7167,  1997,
       19247,  1012,   102]), array([1]))
(array([  101,  2307,  2173,  2005,  6265,  2030,  3347, 27962,  1998,
        5404,  1012,   102]), array([1]))
(array([ 101, 1996, 2047, 2846, 3504, 6429, 1012,  102]), array([1]))
(array([ 101, 2023, 2173, 2001, 2200, 2204, 1012,  102]), array([1]))


In [ ]:
def collate_fn_style(samples):
    input_ids, labels = zip(*samples)
    max_len = max(len(input_id) for input_id in input_ids)
    sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1]

    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True)
    attention_mask = torch.tensor(
        [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
         sorted_indices])
    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])
    labels = torch.tensor(np.stack(labels, axis=0)[sorted_indices])

    return input_ids, attention_mask, token_type_ids, position_ids, labels

In [ ]:
# random seed
random_seed=42
np.random.seed(random_seed)
torch.manual_seed(random_seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def compute_acc(predictions, target_labels):
    return (np.array(predictions) == np.array(target_labels)).mean()

In [ ]:
class Trainer():
    def __init__(self, device, output_path, lr, resume_path):
        self.output_path = output_path
        self.device = device
        self.model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
        self.optimizer = AdamW(self.model.parameters(), lr=lr)
        if resume_path :
            checkpoint = torch.load(resume_path, map_location=device)
            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.start_train_epoch = checkpoint['epoch'] + 1
            self.lowest_valid_loss = checkpoint['lowest_valid_loss']
        else:
            self.start_train_epoch = 0
            self.lowest_valid_loss = 9999.
        self.model.to(self.device)
        for state in self.optimizer.state.values():
            for k, v in state.items():
                if torch.is_tensor(v):
                    state[k] = v.to(device)

    def training(self, train_loader, dev_loader, last_epoch):
        self.model.train()
        for epoch in range(self.start_train_epoch, last_epoch):
            with tqdm(train_loader, unit="batch") as tepoch:
                for iteration, (input_ids, attention_mask, token_type_ids, position_ids, labels) in enumerate(tepoch):
                    tepoch.set_description(f"Epoch {epoch}")
                    input_ids = input_ids.to(self.device)
                    attention_mask = attention_mask.to(self.device)
                    token_type_ids = token_type_ids.to(self.device)
                    position_ids = position_ids.to(self.device)
                    labels = labels.to(self.device, dtype=torch.long)

                    self.optimizer.zero_grad()

                    output = self.model(input_ids=input_ids,
                               attention_mask=attention_mask,
                               token_type_ids=token_type_ids,
                               position_ids=position_ids,
                               labels=labels)

                    loss = output.loss
                    loss.backward()

                    self.optimizer.step()
                
                    tepoch.set_postfix(loss=loss.item())
                    if iteration != 0 and iteration % int(len(train_loader) / 5) == 0:
                        # Evaluate the model five times per epoch
                        with torch.no_grad():
                            self.model.eval()
                            valid_losses = []
                            predictions = []
                            target_labels = []
                            for input_ids, attention_mask, token_type_ids, position_ids, labels in tqdm(dev_loader,
                                                                                                    desc='Eval',
                                                                                                    position=1,
                                                                                                    leave=None):
                                input_ids = input_ids.to(self.device)
                                attention_mask = attention_mask.to(self.device)
                                token_type_ids = token_type_ids.to(self.device)
                                position_ids = position_ids.to(self.device)
                                labels = labels.to(self.device, dtype=torch.long)

                                output = self.model(input_ids=input_ids,
                                           attention_mask=attention_mask,
                                           token_type_ids=token_type_ids,
                                           position_ids=position_ids,
                                           labels=labels)

                                logits = output.logits
                                loss = output.loss
                                valid_losses.append(loss.item())

                                batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
                                batch_labels = [int(example) for example in labels]

                                predictions += batch_predictions
                                target_labels += batch_labels

                        print(epoch)
                        acc = compute_acc(predictions, target_labels)
                        valid_loss = sum(valid_losses) / len(valid_losses)
                        if self.lowest_valid_loss > valid_loss:
                            print('')
                            print('Acc for model which have lower valid loss: ', acc)
                            #torch.save(self.model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/NLP/project/pytorch_model.bin")
                            torch.save({
                                'epoch': epoch,
                                'lowest_valid_loss': self.lowest_valid_loss,
                                'optimizer_state_dict': self.optimizer.state_dict(),
                                'model_state_dict': self.model.state_dict(),
                                }, f'{self.output_path}/checkpoint_epoch_{epoch}.{iteration}.pth')
                            self.lowest_valid_loss = valid_loss
                            print('--------------save checkpoint at epoch : {}--------------'.format(epoch))
                            print('--------------lowest_valid_loss : {}--------------'.format(self.lowest_valid_loss))

In [ ]:
train_batch_size=32
eval_batch_size=64

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=train_batch_size,
                                           shuffle=True, collate_fn=collate_fn_style,
                                           pin_memory=True, num_workers=2)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=eval_batch_size,
                                         shuffle=False, collate_fn=collate_fn_style,
                                         num_workers=2)

# 경로설정
output_path = '/content/drive/MyDrive/Colab Notebooks/NLP/project/checkpoints'
resume_path = '/content/drive/MyDrive/Colab Notebooks/NLP/project/checkpoints/checkpoint_epoch_1.13850.pth'

lr = 5e-6 #5e-5
last_epoch = 5

trainer = Trainer(device,output_path,lr,resume_path)
trainer.training(train_loader, dev_loader, last_epoch)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

2


Epoch 2:  40%|████      | 5542/13852 [09:32<1:41:56,  1.36batch/s, loss=0.0453]

2


Epoch 2:  60%|██████    | 8312/13852 [14:15<1:07:44,  1.36batch/s, loss=0.0117]

...

Epoch 4:  80%|████████  | 11082/13852 [18:47<34:05,  1.35batch/s, loss=0.014]

4


Epoch 4: 100%|█████████▉| 13851/13852 [23:29<00:00,  1.50batch/s, loss=0.00141]

4


Epoch 4: 100%|██████████| 13852/13852 [23:29<00:00,  9.82batch/s, loss=0.00141]


In [ ]:
import pandas as pd
test_df = pd.read_csv('test_no_label.csv')

In [ ]:
test_dataset = test_df['Id']

In [ ]:
def make_id_file_test(tokenizer, test_dataset):
    data_strings = []
    id_file_data = [tokenizer.encode(sent.lower()) for sent in test_dataset]
    for item in id_file_data:
        data_strings.append(' '.join([str(k) for k in item]))
    return data_strings

In [ ]:
test = make_id_file_test(tokenizer, test_dataset)

In [ ]:
test[:10]

['101 2009 1005 1055 1037 2878 2047 3325 1998 2047 26389 2169 2051 2017 2175 1012 102',
 '101 2061 15640 2013 2019 2214 5440 1012 102',
 '101 2009 2003 1996 2087 14469 7273 1999 1996 3028 1012 102',
 '101 2079 2025 3696 1037 10084 2007 2122 2111 1012 102',
 '101 1045 2001 6091 1998 2016 2081 2033 2514 2061 6625 1998 6160 1012 102',
 '101 1996 2069 2518 2057 2363 2008 2001 2980 2001 1996 4157 1012 102',
 '101 2053 1010 2025 1996 3924 2012 2004 2226 1010 1996 3924 1999 3502 2152 1012 102',
 '101 2027 3288 2009 2041 2392 2005 2017 1998 2024 2200 14044 1012 102',
 '101 4606 1996 12043 2106 1050 1005 1056 2130 2113 2129 2000 2147 1996 3274 1012 102',
 '101 2027 2031 2019 6581 4989 1997 25025 2015 2000 5454 2013 1012 102']

In [ ]:
class SentimentTestDataset(object):
    def __init__(self, tokenizer, test):
        self.tokenizer = tokenizer
        self.data = []

        for sent in test:
            self.data += [self._cast_to_int(sent.strip().split())]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample)

In [ ]:
test_dataset = SentimentTestDataset(tokenizer, test)

In [ ]:
def collate_fn_style_test(samples):
    input_ids = samples
    print(input_ids)
    max_len = max(len(input_id) for input_id in input_ids)
    #sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1]
    sorted_indices = list(i for i in range(len(input_ids)))

    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True)
    attention_mask = torch.tensor(
        [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
         sorted_indices])
    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])

    return input_ids, attention_mask, token_type_ids, position_ids

In [ ]:
test_batch_size = 32
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                          shuffle=False, collate_fn=collate_fn_style_test,
                                          num_workers=2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_path = '/content/drive/MyDrive/Colab Notebooks/NLP/project/checkpoints/checkpoint_epoch_1.13850.pth'
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
checkpoint = torch.load(resume_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

with torch.no_grad():
    model.eval()
    predictions = []
    for input_ids, attention_mask, token_type_ids, position_ids in tqdm(test_loader,
                                                                        desc='Test',
                                                                        position=1,
                                                                        leave=None):

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        position_ids = position_ids.to(device)

        output = model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids,
                       position_ids=position_ids)

        logits = output.logits
        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
        predictions += batch_predictions

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

[array([  101,  2009,  1005,  1055,  1037,  2878,  2047,  3325,  1998,
        2047, 26389,  2169,  2051,  2017,  2175,  1012,   102]), array([  101,  2061, 15640,  2013,  2019,  2214,  5440,  1012,   102]), array([  101,  2009,  2003,  1996,  2087, 14469,  7273,  1999,  1996,
        3028,  1012,   102]), array([  101,  2079,  2025,  3696,  1037, 10084,  2007,  2122,  2111,
        1012,   102]), array([ 101, 1045, 2001, 6091, 1998, 2016, 2081, 2033, 2514, 2061, 6625,
       1998, 6160, 1012,  102]), array([ 101, 1996, 2069, 2518, 2057, 2363, 2008, 2001, 2980, 2001, 1996,
       4157, 1012,  102]), array([ 101, 2053, 1010, 2025, 1996, 3924, 2012, 2004, 2226, 1010, 1996,
       3924, 1999, 3502, 2152, 1012,  102]), array([  101,  2027,  3288,  2009,  2041,  2392,  2005,  2017,  1998,
        2024,  2200, 14044,  1012,   102]), array([  101,  4606,  1996, 12043,  2106,  1050,  1005,  1056,  2130,
        2113,  2129,  2000,  2147,  1996,  3274,  1012,   102]), array([  101,  2027,  2031


Test:   3%|▎         | 1/32 [00:00<00:06,  5.12it/s]

[array([ 101, 1045, 2097, 2022, 2183, 2067, 1998, 9107, 2023, 2307, 2173,
        999,  102]), array([ 101, 2053, 2655, 1010, 2053, 2498, 1012,  102]), array([  101,  2190,  6350,  1045,  2018,  2006,  2026,  3522,  4440,
        2000,  3660, 15145,  1012,   102]), array([  101,  2074,  2253,  2067,  2000,  2131,  1996,  4373, 12824,
       10203,  1998,  3561,  1012,   102]), array([  101,  2307,  2833,  1010,  2307,  2326,  1010,  2037, 18640,
        2064,  2025,  2022,  3786,  1012,   102]), array([ 101, 1045, 3984, 2016, 2001, 1050, 1005, 1056, 3407, 2008, 2057,
       2020, 4851, 1996, 7597, 1012,  102]), array([  101,  1045,  2052,  2196, 28667,  9006, 10497,  3087,  2000,
        2444,  2182,  1012,   102]), array([ 101, 2023, 2173, 2003, 3968, 6292, 1998, 4569, 1012,  102]), array([ 101, 2023, 2173, 2003, 3458, 1037, 4485, 4920, 1012,  102]), array([  101,  1045,  2036,  2131, 18064,  2182,  2043,  1045,  2031,
        1037,  4086, 11868,  1012,   102]), array([ 101, 1045, 201


Test:  12%|█▎        | 4/32 [00:00<00:01, 15.44it/s]

...


Test:  66%|██████▌   | 21/32 [00:00<00:00, 27.87it/s]

[array([  101,  2467,  4840,  1010,  6638,  1010,  1998, 12090,  1012,
         102]), array([  101,  1045,  3641, 20548,  7852,  1998, 10768,  4779, 16835,
        2638, 19423, 24857,  2007, 11546,  1012,   102]), array([  101,  3046,  2505,  2006,  1996, 12183,  1998,  2017,  2097,
        2022,  5580,  2017,  2234,  1999,   999,   102]), array([ 101, 1045, 8969, 2023, 1998, 2002, 2253, 2067, 2000, 2202, 2729,
       1997, 1996, 3021, 1012,  102]), array([ 101, 2968, 2515, 2025, 4025, 2000, 2729, 1012,  102]), array([  101,  5379,  1998, 18066,  2007,  1037,  4569,  7224,  1998,
       27547,  2833,  1012,   102]), array([  101,  2431,  1997,  2026,  2132,  2001,  2058, 13995,  1012,
         102]), array([ 101, 2027, 5338, 2033, 2005, 2147, 2025, 2589, 1010, 1998, 3033,
       2025, 5361, 1012,  102]), array([ 101, 2017, 1005, 2222, 2156, 2339, 2320, 2017, 2131, 2045, 1012,
        102]), array([ 101, 2074, 6659, 2003, 2035, 1045, 2064, 2360, 1012,  102]), array([ 101, 2307, 2326, 2


Test:  78%|███████▊  | 25/32 [00:00<00:00, 29.59it/s]

[array([  101,  2200, 14044,  1010,  7570, 13102,  6590,  3468,  1010,
        3716,  3085,  1010,  1998, 12367,  8082,  1012,   102]), array([ 101, 2026, 2684, 2363, 2307, 2729, 1012,  102]), array([ 101, 2057, 2409, 2068, 2000, 5293, 2009, 1010, 2057, 2106, 1050,
       1005, 1056, 2215, 2000, 3524, 1012,  102]), array([ 101, 2941, 1010, 2074, 2562, 3788, 1012,  102]), array([  101,  2021,  2009,  2001, 19424,  1998,  2980,  1999,  2045,
        1012,   102]), array([  101,  2035,  2016,  2106,  2001,  2507,  2033,  1996,  2448,
        2105,  1998,  9828,  1998, 18667,  2673,  1012,   102]), array([ 101, 2200, 5379, 3095, 1998, 3208, 1012,  102]), array([ 101, 2027, 2036, 2024, 1996, 2190, 3976, 1999, 1996, 2181, 1012,
        102]), array([ 101, 1996, 2396, 1999, 1996, 8975, 2369, 2009, 2003, 4658, 2205,
        999,  102]), array([ 101, 2498, 2008, 2569, 2055, 2023, 2173, 1012,  102]), array([  101,  2058,  3597, 23461,  2061,  6649,  2008,  2009,  2001,
        1996, 18700,  1997


Test:  91%|█████████ | 29/32 [00:01<00:00, 30.51it/s]
                                                     

In [ ]:
test_df['Category'] = predictions

In [ ]:
test_df.to_csv('submission.csv', index=False)